In [1]:
import logging
import os
import pymongo
import cobrakbase
import time
import biosapi

logger = logging.getLogger(__name__)
%run ../../scripts/bios_utils.py

cobrakbase 0.2.7


In [2]:
%run ../utils.py

In [3]:
%run ../annotation_ortholog.py
%run ../bios_mock.py

In [4]:
CACHE_BASE_FOLDER = '/Users/fliu/workspace/jupyter/python3/annotation-server/data/'
kbase = cobrakbase.KBaseAPI()
bios = biosapi.BIOS()

In [5]:
mongo_client = pymongo.MongoClient("192.168.1.15")
database = mongo_client['KO']

In [6]:
collection_genome = database['genome']
collection_ko = database['ko']

In [64]:
t_tax = {}
tax_t = {}
t_value = {}
for doc in collection_genome.find():
    a, b= doc['taxonomy'].split(':')
    t_tax[doc['_id']] = b
    tax_t[b] = doc['_id']
    t_value[doc['_id']] = doc

In [79]:
kegg_scan_genomes = set()
for ncbi_tax in taxa_id_to_genome_id:
    genome_id = taxa_id_to_genome_id[ncbi_tax]
    if ncbi_tax in tax_t:
        taxa = genome_taxa[genome_id]
        doc = t_value[tax_t[ncbi_tax]]
        kegg_name_token = doc['name'].split(',')[0]
        kegg_scan_genomes.add(kegg_name_token)
        print(genome_id, ncbi_tax, tax_t[ncbi_tax], kegg_name_token)
        #print(taxa.scientific_lineage, taxa.scientific_name)
        #print(doc['lineage'], doc['definition'])
    else:
        #print(genome_id, ncbi_tax, '?')
        #print(taxa.scientific_lineage)
        #print('?')
        pass
    #print()

GCF_000002515.2 284590 T01025 kla
GCF_000209165.1 322104 T01023 pic
GCF_000002525.2 284591 T01033 yli
GCF_000184455.2 510516 T01018 aor
GCF_000027005.1 644223 T01092 ppa
GCF_000002545.3 284593 T01024 cgr
GCF_000091025.4 284811 T00167 ago
GCF_000006335.2 294747 T01115 ctp
GCF_000226395.1 500485 T01091 pcs
Alternaria_alternata 5599 T05291 aalt
GCF_000182925.2 367110 T01034 ncr
GCF_000146045.2 559292 T00005 sce


In [81]:
doc = collection_ko.find_one()

In [102]:
ko_genome_genes = {}
for doc in collection_ko.find():
    ko_genome_genes[doc['_id']] = {}
    for g in doc['genes_raw']:
        kegg_genome_token, b = g.split(':', maxsplit=1)
        kegg_genome_token = kegg_genome_token.lower()
        if kegg_genome_token in kegg_scan_genomes:
            ko_genome_genes[doc['_id']][kegg_genome_token] = b

In [108]:
ortholog = pangenome.get_ortholog_by_id('cluster934')

In [154]:
genome_genes_ko = {}
for genome_id in kegg_scan_genomes:
    genome_genes_ko[genome_id] = {}
for ko in ko_genome_genes:
    for genome_id in ko_genome_genes[ko]:
        gene_str = ko_genome_genes[ko][genome_id]
        for gene_tk in gene_str.split():
            gene_id = gene_tk
            gene_id = gene_id.split('(')[0]
            if not gene_id in genome_genes_ko[genome_id]:
                genome_genes_ko[genome_id][gene_id] = set()
            genome_genes_ko[genome_id][gene_id].add(ko)

In [171]:
def get_ko_map(ortholog):
    ko_map = {}
    gene_fail = {}
    for o in ortholog['orthologs']:
        gene_id, x, genome_ref = o
        genome_id = pangenome.genome_ref_object_info[genome_ref].id
        ncbi_tax = str(genome_taxa[genome_id].taxonomy_id)
        if ncbi_tax in tax_t:
            doc = t_value[tax_t[ncbi_tax]]
            kegg_name_token = doc['name'].split(',')[0]
            if gene_id in genome_genes_ko[kegg_name_token]:
                for ko in genome_genes_ko[kegg_name_token][gene_id]:
                    if ko not in ko_map:
                        ko_map[ko] = set()
                    ko_map[ko].add((gene_id, genome_id))
                #print(gene_id, genome_id, ncbi_tax, kegg_name_token, genome_genes_ko[kegg_name_token][gene_id])
            else:
                #print(gene_id, genome_id, ncbi_tax, kegg_name_token, '?')
                pass
        else:
            #print(gene_id, genome_id, ncbi_tax, '?', '?')
            pass
    return ko_map

In [ ]:
for ortholog_id in pangenome.ortholog_ids:
    ortholog = pangenome.get_ortholog_by_id(ortholog_id)
    ko_map = get_ko_map(ortholog)
    ortholog['ko'] = ko_map

In [52]:
taxa_id_to_genome_id = {}
for genome_id in genome_taxa:
    taxa_id_to_genome_id[str(genome_taxa[genome_id].taxonomy_id)] = genome_id

In [8]:
#ortho = kbase.get_object('FungalTemplateAndRepresentativeSetOrthosV0', 'filipeliu:narrative_1575329472745')
pangenome = kbase.get_from_ws('TemplateGenomeSet_FilipeTestGenomeSetOrthoLogs', 'janakakbase:narrative_1570052138482')
pangenome.api = kbase

In [12]:
print(pangenome.id)
print(pangenome.name)
print(pangenome.info.type)
print(pangenome.info.version)

TemplateGenomeSet_FilipeTestGenomeSetOrthoLogs
TemplateGenomeSet_FilipeTestGenomeSetOrthoLogs
KBaseGenomes.Pangenome
1


In [19]:
ortholog = pangenome.get_ortholog_by_id('cluster1')
print(ortholog.keys())
#ortholog

dict_keys(['function', 'id', 'orthologs'])


In [20]:
len(pangenome.ortholog_ids)

82012

In [91]:
genome_id_to_alias = {
    'Mucor_circinelloides_CBS277.49_v2.0' : 'Mucor',
'Lipst1_1' : 'Lipomyces',
'Jaaar1' : 'Jaapia',
'Hydpi2' : 'Hydnomerulius',
'Hanva1_1' : 'Hanseniaspora',
'Gymlu1' : 'Gymnopus',
'Gloin1' : 'Rhizophagus',
'GCF_000226395.1' : 'Penicillium',
'GCF_000209165.1' : 'S. stipitis',
'GCF_000184455.2' : 'A. oryzae',
'GCF_000182925.2' : 'Neurospora',
'GCF_000149615.1' : 'A. terreus',
'GCF_000146045.2' : 'S. cerevisiae',
'GCF_000091025.4' : 'E. gossypii',
'GCF_000027005.1' : 'K. phaffii',
'GCF_000006335.2' : 'C. tropicalis',
'GCF_000002545.3' : 'C. glabrata',
'GCF_000002525.2' : 'Yarrowia',
'GCF_000002515.2' : 'K. lactis',
'Ganoderma_sp.' : 'Ganoderma',
'Fibsp1' : 'Fibularhizoctonia',
'Exigl1' : 'Exidia',
'Eurhe1' : 'Aspergillus ruber',
'Alternaria_alternata' : 'Alternaria',
}

In [116]:
data = {
    'id' : [],
    'ko' : [],
    'function' : []
}
genome_ids = set()
for genome_ref in pangenome.genome_ref_object_info:
    info = pangenome.genome_ref_object_info[genome_ref]
    data[info.id] = []
    genome_ids.add(info.id)
    #print(genome_id_to_alias[info.id])

In [117]:
def add_row(ortholog, data):
    genome_id_to_genes = {}
    for genome_id in genome_ids:
        genome_id_to_genes[genome_id] = set()
    for o_gene in ortholog.data['orthologs']:
        genome_ref = o_gene[2]
        genome_id = pangenome.genome_ref_object_info[genome_ref].id
        genome_id_to_genes[genome_id].add(o_gene[0])
        
    data['id'].append(ortholog.id)
    data['function'].append(ortholog.function)
    data['ko'].append('')
    for genome_id in genome_ids:
        data[genome_id].append(';'.join(genome_id_to_genes[genome_id]))
        


In [118]:
for ortholog_id in pangenome.ortholog_ids:
    ortholog = OrthologFamily(pangenome.get_ortholog_by_id(ortholog_id))
    add_row(ortholog, data)

In [119]:
df = pd.DataFrame(data)

In [121]:
df.to_csv('/Users/fliu/workspace/jupyter/otable.tsv', sep='\t')

In [ ]:
#PYK cluster934

In [127]:
model = kbase.get_object('fungal_template_model_ontology', 'filipeliu:narrative_1548978605026')


In [128]:
kbase.get_object_info_from_ref(model['genome_ref']).id

'FungalTemplate.genome'

In [126]:
kbase.save_model('fungal_template_model_ontology_sce', 'filipeliu:narrative_1548978605026', 'KBaseFBA.FBAModel', model)

[[353,
  'fungal_template_model_ontology_sce',
  'KBaseFBA.FBAModel-12.0',
  '2020-08-19T04:16:59+0000',
  1,
  'filipeliu',
  40036,
  'filipeliu:narrative_1548978605026',
  '507826aa6e7c84beb1a7699cfc9a1d3a',
  299576,
  {'Number gapgens': '0',
   'Type': 'GenomeScale',
   'Number gapfills': '0',
   'Source ID': 'fungal_merge_model',
   'Number biomasses': '0',
   'Number compartments': '4',
   'Genome': '40036/286/1',
   'Source': 'KBase',
   'Number compounds': '120',
   'Number reactions': '119',
   'Name': 'fungal_merge_model'}]]

In [40]:
genome_taxa = {}
for genome_id in genomes:
    genome_taxa[genome_id] = kbase.get_from_ws(genomes[genome_id].data['taxon_ref'])

In [29]:
genomes = {}
genome_folder = '/Users/fliu/workspace/jupyter/data/genomes/fungi/'
for genome_ref in pangenome.genome_ref_object_info:
    genome_id = pangenome.genome_ref_object_info[genome_ref].id
    print(genome_ref, genome_id)
    #ref = kbase.get_object_info_from_ref(genome_ref)
    #o = kbase.get_object(ref.id, ref.workspace_id)
    #with open(genome_folder + genome_id + '.json', 'w') as fh:
    #    fh.write(json.dumps(o))
    #break
    with open(genome_folder + genome_id + '.json', 'r') as fh:
        genomes[genome_id] = cobrakbase.core.KBaseGenome(json.loads(fh.read()))

49164/18/1 GCF_000002515.2
49164/9/1 GCF_000149615.1
49164/14/1 GCF_000209165.1
49164/97/7 Gloin1
49164/15/1 GCF_000002525.2
49164/16/1 Mucor_circinelloides_CBS277.49_v2.0
49164/17/1 GCF_000184455.2
49164/12/1 GCF_000027005.1
49164/10/1 GCF_000002545.3
49164/13/1 GCF_000091025.4
49164/11/1 GCF_000006335.2
49164/6/2 GCF_000226395.1
49164/105/1 Alternaria_alternata
49164/100/7 Eurhe1
49164/5/2 GCF_000182925.2
49164/104/1 Ganoderma_sp.
49164/101/7 Fibsp1
49164/98/6 Hanva1_1
49164/4/3 GCF_000146045.2
49164/99/7 Gymlu1
49164/95/6 Lipst1_1
49164/102/7 Exigl1
49164/103/6 Hydpi2
49164/96/6 Jaaar1


In [11]:
pangenome.data.keys()

dict_keys(['genome_refs', 'id', 'name', 'orthologs', 'type'])

In [15]:
print(pangenome.id)
print(pangenome.name)
print(pangenome.otype)
print(pangenome.otype_version)

FungalTemplateAndRepresentativeSetOrthosV0
FungalTemplateAndRepresentativeSetOrthosV0
None
None


In [21]:
bios_mock, MODEL_CMP_MAPPING, MODEL_CPD_MAPPING, MODEL_RXN_MAPPING, MODEL_RXN_GPR = load_cache_data('/Users/fliu/workspace/jupyter/python3/annotation-server/data/')

In [22]:
bios_mock.get_model_reactions_by_database_id('iMM904', 'rxn00543', 'ModelSeedReaction')

[(<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd521278d10>,
  'mock:5'),
 (<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd521278cd0>,
  'mock:5'),
 (<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd521278c90>,
  'mock:5'),
 (<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd521278c50>,
  'mock:5')]

In [8]:
bios.get_model_reactions_by_database_id('iMM904', 'rxn00543', 'ModelSeedReaction')

[(<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd6a2d86650>,
  'fliu:4;system:3'),
 (<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd6a2d86690>,
  'fliu:4;system:3'),
 (<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd6a2d866d0>,
  'fliu:4;system:3'),
 (<biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7fd6a2d86710>,
  'fliu:4;system:3')]

In [9]:
annotation_ortholog = build_annotation_ortholog(kbase, CACHE_BASE_FOLDER, bios_mock)

load local 49164/31/1
Rhizophagus_irregularis janakakbase:narrative_1570052138482
load kbase 49164/29/1
GCF_000320785.1 janakakbase:narrative_1570052138482
load local 49164/18/1
GCF_000002515.2 janakakbase:narrative_1570052138482
load local 49164/28/1
Tuber_melanosporum_Mel28 janakakbase:narrative_1570052138482
load kbase 49164/25/1
GCF_000328475.2 janakakbase:narrative_1570052138482
load local 49164/9/1
GCF_000149615.1 janakakbase:narrative_1570052138482
load local 49164/27/1
Cryptococcus_neoformans_var._grubii_H99 janakakbase:narrative_1570052138482
load local 49164/34/1
Piromyces_finnis janakakbase:narrative_1570052138482
load local 49164/14/1
GCF_000209165.1 janakakbase:narrative_1570052138482
load local 49164/15/1
GCF_000002525.2 janakakbase:narrative_1570052138482
load local 49164/16/1
Mucor_circinelloides_CBS277.49_v2.0 janakakbase:narrative_1570052138482
load local 49164/39/1
Mucor_circinelloides_f_lusitanicus_CBS277_49 janakakbase:narrative_1570052138482
load local 49164/12/1


GCF_000146045.2 janakakbase:narrative_1570052138482
49164/31/1 Gloin1 Rhizophagus irregularis
49164/29/1 GCF_000320785.1 Rhodotorula toruloides NP11
49164/18/1 GCF_000002515.2 Kluyveromyces lactis NRRL Y-1140
49164/28/1 GCF_000151645.1 Tuber melanosporum Mel28
49164/25/1 GCF_000328475.2 Ustilago maydis 521
49164/9/1 GCF_000149615.1 Aspergillus terreus NIH2624
49164/27/1 GCF_000149245.1 Cryptococcus neoformans var. grubii H99
49164/34/1 Pirfi3 Piromyces finnis
49164/14/1 GCF_000209165.1 Scheffersomyces stipitis CBS 6054
49164/15/1 GCF_000002525.2 Yarrowia lipolytica CLIB122
49164/16/1 Mucor_circinelloides_CBS277.49_v2.0 unknown_taxon
49164/39/1 Mucor_circinelloides_f_lusitanicus_CBS277_49 Mucor circinelloides f. lusitanicus CBS 277.49
49164/12/1 GCF_000027005.1 Komagataella phaffii GS115
49164/36/1 Aspergillus_nidulans Aspergillus nidulans FGSC A4
49164/26/1 Lacbi2 Laccaria bicolor
49164/10/1 GCF_000002545.3 Candida glabrata CBS 138
49164/13/1 GCF_000091025.4 Eremothecium gossypii ATCC 

In [23]:
annotation_ortholog.bios = bios_mock
annotation_ortholog.model_rxn_mapping = MODEL_RXN_MAPPING
annotation_ortholog.model_cpd_mapping = MODEL_CPD_MAPPING
annotation_ortholog.model_rxn_gpr = MODEL_RXN_GPR

In [24]:
annotation_ortholog.get_orthologs_from_seed_rxn_id3('rxn09528')

iNL895
[(<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd5209abb10>, 'mock:5'), (<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd5207c4710>, 'mock:5'), (<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd57485b050>, 'mock:5')]
iCT646
[(<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd574859f90>, 'mock:5')]
iMM904
[(<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd520547d90>, 'mock:5'), (<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd5207b4dd0>, 'mock:5'), (<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd52079d610>, 'mock:5')]
iTO977
[]
iSS884
[(<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd521275cd0>, 'mock:5')]
iLC915
[(<biosapi.core.model.bios_model_reaction.BiosModelReaction object at 0x7fd5207912d0>, 'mock:5')]
iWV1213
[(<biosapi.core.model.bios_model_reaction.BiosModelReaction object at

({('GCF_000002515.2', 'KLLA0A01947g', 'cluster1340'),
  ('GCF_000002515.2', 'KLLA0A01991g', 'cluster1633'),
  ('GCF_000002515.2', 'KLLA0A04543g', 'cluster2081'),
  ('GCF_000002515.2', 'KLLA0A09031g', 'cluster81'),
  ('GCF_000002515.2', 'KLLA0B04752g', 'cluster1929'),
  ('GCF_000002515.2', 'KLLA0B06886g', 'cluster1551'),
  ('GCF_000002515.2', 'KLLA0B07843g', 'cluster8'),
  ('GCF_000002515.2', 'KLLA0C05170g', 'cluster2081'),
  ('GCF_000002515.2', 'KLLA0C07491g', 'cluster260'),
  ('GCF_000002515.2', 'KLLA0C10384g', 'cluster1603'),
  ('GCF_000002515.2', 'KLLA0C15917g', 'cluster3238'),
  ('GCF_000002515.2', 'KLLA0C17446g', 'cluster2582'),
  ('GCF_000002515.2', 'KLLA0D04224g', 'cluster8'),
  ('GCF_000002515.2', 'KLLA0D04400g', 'cluster2318'),
  ('GCF_000002515.2', 'KLLA0D09152g', 'cluster1081'),
  ('GCF_000002515.2', 'KLLA0D09559g', 'cluster8'),
  ('GCF_000002515.2', 'KLLA0D10703g', 'cluster435'),
  ('GCF_000002515.2', 'KLLA0D19041g', 'cluster8'),
  ('GCF_000002515.2', 'KLLA0E02553g', 'clust

In [63]:
import json
rxn_mapping_cache = None
with open('/Users/fliu/workspace/jupyter/python3/annotation-server/data/rxn_mapping_cache4.json', 'r') as fh:
    rxn_mapping_cache = json.loads(fh.read())

In [64]:
rxn_mapping_cache.keys()

dict_keys(['iAL1006', 'iCT646', 'iLC915', 'iMM904', 'iRL766', 'iSS884', 'iTO977', 'yeast_6.06', 'yeast_7.6', 'iJDZ836', 'iWV1213', 'iJO1366', 'iCac802', 'iAF1260', 'iML1515', 'iMA871', 'iOD907', 'iNL895', 'iJL1454', 'iNX804', 'iWV1314', 'MitoCore'])

In [27]:
mrxn.get_stoichiometry()

{'M_nad_c': -1.0,
 'M_etoh_c': -1.0,
 'M_nadh_c': 1.0,
 'M_acald_c': 1.0,
 'M_h_c': 1.0}

In [25]:
mrxn = res['R_ALCD2x_copy1']

In [16]:
res = annotation_ortholog.get_model_reaction_from_seed_id('iMM904', 'rxn00543', 5)
res

{'R_ALCD2ir': <biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7ff54d15d810>,
 'R_ALCD2irm': <biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7ff54d15d950>,
 'R_ALCD2x_copy1': <biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7ff54d15d910>,
 'R_ALCD2x_copy2': <biosapi.core.model.bios_model_reaction.BiosModelReaction at 0x7ff54d15d990>}

In [42]:
for r in model.reactions:
    sid = r.get_original_id()
    if 'imported_gpr' in r.data:
        #print(sid)
        break
    #print(sid)
r.data

{'aliases': [],
 'dblinks': {},
 'direction': '=',
 'id': 'r-4021_z4',
 'imported_gpr': '',
 'maxforflux': 1000.0,
 'maxrevflux': 0.0,
 'modelReactionProteins': [],
 'modelReactionReagents': [{'coefficient': -1.0,
   'modelcompound_ref': '~/modelcompounds/id/s-3026_z4'},
  {'coefficient': 0.679012345679012,
   'modelcompound_ref': '~/modelcompounds/id/cpd11677_z4'}],
 'modelcompartment_ref': '~/modelcompartments/id/z4',
 'name': 'isa triglyceride for TAG (1-18:1, 2-18:1, 3-16:1)',
 'pathway': 'entire model',
 'probability': 1.0,
 'protons': 1.0,
 'reaction_ref': '~/template/reactions/id/rxn00000_c',
 'string_attributes': {'original_id': 'r_4021'}}

In [18]:
#annotation_ortholog.get_orthologs_from_seed_rxn_id2('rxn00543')

In [72]:
with open('/Users/fliu/workspace/jupyter/python3/annotation-server/data/model_rxn_gpr_cache.json', 'w') as fh:
    fh.write(json.dumps(model_rxn_gpr))

In [54]:
model_rxn_gpr = {}
for model_id in annotation_ortholog.models:
    model_rxn_gpr[model_id] = {}
    model = annotation_ortholog.models[model_id]
    for r in model.reactions:
        sid = r.get_original_id()
        gpr_exp = ''
        if 'imported_gpr' in r.data:
            gpr_exp = r.data['imported_gpr']
        if model_id == 'iMA871':
            gpr_exp = gpr_exp.replace('; ', ' or ')
        if model_id == 'iCT646' and sid == 'R_FAS100':
            gpr_exp = 'CTRG_02936 or (CTRG_05241 and CTRG_02501)'
            print(r.data['imported_gpr'], '->', gpr_exp)
        model_rxn_gpr[model_id][sid] = gpr_exp

CTRG_02936 OR (CTRG_05241 ANDCTRG_02501) -> CTRG_02936 or (CTRG_05241 and CTRG_02501)


In [73]:
def get_model_rxn_grp2(aa, seed_rxn_id, compartment=None, standard_compartment=True):
    model_rxn_grp = {}
    for model_id in aa.models:
        #print(model_id)
        #model_rxn_grp[model_id] = get_model_rxn_grp3(aa, model_id, seed_rxn_id, compartment, standard_compartment)
        model_rxn_grp[model_id] = get_model_reaction_from_seed_id(aa, model_id, seed_rxn_id, 5)
    return model_rxn_grp

def get_orthologs_from_seed_rxn_id3(aa, seed_rxn_id, compartment=None, standard_compartment=True):
    model_rxn_grp = get_model_rxn_grp2(annotation_ortholog, seed_rxn_id, compartment, standard_compartment)
    genome_match = {}
    all_orthologs = set()
    matched_orthologs = set()
    for model_id in model_rxn_grp:
        genome_id = aa.model_to_genome[model_id]
        if genome_id in aa.genome_id_to_ref:
            if genome_id not in genome_match:
                genome_match[genome_id] = {}
            genome_match[genome_id][model_id] = []
            genome = aa.ref_to_genome[aa.genome_id_to_ref[genome_id]]
            for rxn_id in model_rxn_grp[model_id]:
                gpr_exp = get_gpr(model_id, model_rxn_grp[model_id][rxn_id])
                #print(rxn_id, gpr_exp)
                genes = aa.get_genes(gpr_exp)
                exp_match = {
                    'rxn_id': rxn_id,
                    'gpr': gpr_exp,
                    'genes': {}
                }

                for gene_id in genes:
                    features = list(filter(lambda x: x['id'] == gene_id, genome.features))
                    if len(features) == 1:
                        ortholog = aa.get_ortholog(genome_id, gene_id)
                        matched_orthologs.add((genome_id, gene_id, ortholog['id']))
                        for o in ortholog['orthologs']:
                            o_genome_id = aa.ref_to_genome[o[2]].id
                            if o_genome_id in aa.model_to_genome.values():
                                all_orthologs.add((o_genome_id, o[0], ortholog['id']))
                        gene_function = '?' if not 'function' in features[0] else features[0]['function']
                        #print(features[0]['id'], gene_function, ortholog['id'])
                        exp_match['genes'][gene_id] = {
                            'ortholog_id' : ortholog['id'],
                            'function' : gene_function
                        }
                    else:
                        print('!', gene_id, len(features))
                genome_match[genome_id][model_id].append(exp_match)
    return all_orthologs, matched_orthologs, genome_match
get_orthologs_from_seed_rxn_id3(annotation_ortholog, 'rxn01100')

! AEL038C 0


({('GCF_000002515.2', 'KLLA0A04631g', 'cluster156'),
  ('GCF_000002515.2', 'KLLA0A06886g', 'cluster17'),
  ('GCF_000002515.2', 'KLLA0A06930g', 'cluster17'),
  ('GCF_000002515.2', 'KLLA0A10813g', 'cluster17'),
  ('GCF_000002515.2', 'KLLA0A11011g', 'cluster946'),
  ('GCF_000002515.2', 'KLLA0A11770g', 'cluster17'),
  ('GCF_000002515.2', 'KLLA0B00352g', 'cluster89'),
  ('GCF_000002515.2', 'KLLA0B06798g', 'cluster262'),
  ('GCF_000002515.2', 'KLLA0B12826g', 'cluster1398'),
  ('GCF_000002515.2', 'KLLA0B13233g', 'cluster2036'),
  ('GCF_000002515.2', 'KLLA0C01606g', 'cluster17'),
  ('GCF_000002515.2', 'KLLA0C18931g', 'cluster5544'),
  ('GCF_000002515.2', 'KLLA0D04950g', 'cluster2196'),
  ('GCF_000002515.2', 'KLLA0D07073g', 'cluster1047'),
  ('GCF_000002515.2', 'KLLA0D15015g', 'cluster2780'),
  ('GCF_000002515.2', 'KLLA0D19941g', 'cluster12'),
  ('GCF_000002515.2', 'KLLA0E02817g', 'cluster2838'),
  ('GCF_000002515.2', 'KLLA0E03829g', 'cluster191'),
  ('GCF_000002515.2', 'KLLA0E09637g', 'cluster

In [25]:
annotation_ortholog.get_orthologs_from_seed_rxn_id2('rxn00543', 'c', False)

AttributeError: 'BiosModelReaction' object has no attribute 'data'

In [16]:
bios = biosapi.BIOS()
#bios.get_model_reaction('iMM904', 'R_PYK')

In [23]:
def get_model_rxn_grp(annotation_ortholog, seed_rxn_id, compartment = None, standard_compartment = True):
    model_rxn_grp = {}
    for model_id in annotation_ortholog.models:
        model_rxn_grp[model_id] = {}
        if model_id in annotation_ortholog.model_rxn_mapping:
            for rxn_id in annotation_ortholog.model_rxn_mapping[model_id]:
                if len(annotation_ortholog.model_rxn_mapping[model_id][rxn_id]) > 0:
                    rxn = annotation_ortholog.bios_get_model_reaction(model_id, rxn_id)
                    cmp = annotation_ortholog.get_rxn_compartment(rxn, model_id)
                    seed_id = annotation_ortholog.get_reaction_mapping(model_id, rxn.id)
                    if seed_rxn_id in seed_id:
                        if compartment == None:
                            kbase_rxn = annotation_ortholog.get_kbase_reaction(model_id, rxn.id)
                            if not kbase_rxn == None:
                                model_rxn_grp[model_id][rxn.id] = kbase_rxn
                            else:
                                print('!!!!')
                        else:
                            model_rxn_grp[model_id][rxn.id] = rxn
                #print(rxn_id, get_rxn_compartment(rxn, model_id))
    return model_rxn_grp

def get_model_rxn_grp2(annotation_ortholog, model_id, seed_rxn_id, compartment = None, standard_compartment = True):
    model_rxn_grp = {}
    for model_id in annotation_ortholog.models:
        model_rxn_grp[model_id] = {}
        if model_id in annotation_ortholog.model_rxn_mapping:
            for rxn_id in annotation_ortholog.model_rxn_mapping[model_id]:
                if len(annotation_ortholog.model_rxn_mapping[model_id][rxn_id]) > 0:
                    rxn = annotation_ortholog.bios_get_model_reaction(model_id, rxn_id)
                    cmp = annotation_ortholog.get_rxn_compartment(rxn, model_id)
                    seed_id = annotation_ortholog.get_reaction_mapping(model_id, rxn.id)
                    if seed_rxn_id in seed_id:
                        if compartment == None:
                            kbase_rxn = annotation_ortholog.get_kbase_reaction(model_id, rxn.id)
                            if not kbase_rxn == None:
                                model_rxn_grp[model_id][rxn.id] = kbase_rxn
                            else:
                                print('!!!!')
                        else:
                            model_rxn_grp[model_id][rxn.id] = rxn
                #print(rxn_id, get_rxn_compartment(rxn, model_id))
    return model_rxn_grp
get_model_rxn_grp2(annotation_ortholog, "iMM904", "rxn00024")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [22]:
%run ../bios_mock.py
bios = BIOS_MOCK(CACHE_BASE_FOLDER + '/bios_cache_fungi.json')

In [24]:
annotation_ortholog = build_annotation_ortholog(kbase, CACHE_BASE_FOLDER, bios)

GCF_000226395.1 janakakbase:narrative_1570052138482
GCF_000182925.2 janakakbase:narrative_1570052138482
GCF_000146045.2 janakakbase:narrative_1570052138482
Encephalitozoon_hellem_ATCC_50504 janakakbase:narrative_1570052138482


Schizophyllum_commune_H4-8 janakakbase:narrative_1570052138482
49164/6/1 GCF_000226395.1 Penicillium rubens Wisconsin 54-1255
49164/5/1 GCF_000182925.2 Neurospora crassa OR74A
49164/4/1 GCF_000146045.2 Saccharomyces cerevisiae S288c
49164/3/1 GCF_000277815.2 Encephalitozoon hellem ATCC 50504
49164/2/1 GCF_000143185.1 Schizophyllum commune H4-8


In [25]:
annotation_ortholog.model_rxn_mapping = model_rxn_mapping
annotation_ortholog.model_cpd_mapping = model_cpd_mapping

In [34]:
all_orthologs, matched_orthologs, genome_match = annotation_ortholog.get_orthologs_from_seed_rxn_id2('rxn00148')

YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614
YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614
Pc18g06000 hypothetical protein cluster614
YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614
YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614


In [ ]:
annotation_orth.process_data2

In [ ]:
'' ''

In [52]:
def get_ortholog_from_gene(annotation_ortholog, gene_id, genome_id):
    ortholog = annotation_ortholog.get_ortholog(genome_id, gene_id)
    return ortholog

genome_ref = annotation_ortholog.genome_id_to_ref[genome_id]

KeyError: 'GCF_000146045.2l'

In [53]:
gene_id = 'YKL192C'
genome_id = 'GCF_000146045.2'
res = annotation_ortholog.get_ortholog(genome_id, gene_id)
column_data = {}
if not res == None:
    ortholog_id = res['id']
    column_data = annotation_ortholog.process_data2({
        (genome_id, gene_id, ortholog_id)
    }, {})
column_data

{'gene_reaction': {'GCF_000226395.1': {'Pc18g03290': []},
  'GCF_000182925.2': {'NCU05008': []},
  'GCF_000146045.2': {'YKL192C': ['r_2143',
    'r_2142',
    'r_0721',
    'r_2144',
    'r_0099']},
  'GCF_000143185.1': {'SCHCODRAFT_64587': []}},
 'orthologs': {'cluster3465': {'GCF_000226395.1': {'Pc18g03290': 'hypothetical protein'},
   'GCF_000182925.2': {'NCU05008': 'acp-1'},
   'GCF_000146045.2': {'YKL192C': 'acyl carrier protein'},
   'GCF_000143185.1': {'SCHCODRAFT_64587': '?'}}},
 'genomes': {}}

In [39]:
all_orthologs

{('GCF_000146045.2', 'YAL038W', 'cluster614'),
 ('GCF_000146045.2', 'YOR347C', 'cluster614'),
 ('GCF_000182925.2', 'NCU06075', 'cluster614'),
 ('GCF_000226395.1', 'Pc18g06000', 'cluster614')}

{'gene_reaction': {'GCF_000226395.1': {'Pc18g03290': []},
  'GCF_000182925.2': {'NCU05008': []},
  'GCF_000146045.2': {'YKL192C': ['r_2143',
    'r_2142',
    'r_0721',
    'r_2144',
    'r_0099']},
  'GCF_000143185.1': {'SCHCODRAFT_64587': []}},
 'orthologs': {'cluster3465': {'GCF_000226395.1': {'Pc18g03290': 'hypothetical protein'},
   'GCF_000182925.2': {'NCU05008': 'acp-1'},
   'GCF_000146045.2': {'YKL192C': 'acyl carrier protein'},
   'GCF_000143185.1': {'SCHCODRAFT_64587': '?'}}},
 'genomes': {}}